## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Current Description
0,0,Saint-Philippe,RE,-21.3585,55.7679,71.19,77,39,13.11,2022-08-15 16:11:51,scattered clouds
1,1,Yellowknife,CA,62.4560,-114.3525,67.19,67,0,7.49,2022-08-15 16:11:51,clear sky
2,2,Namtsy,RU,62.7161,129.6658,55.49,98,48,5.03,2022-08-15 16:11:52,scattered clouds
3,3,Busselton,AU,-33.6500,115.3333,65.03,93,100,27.04,2022-08-15 16:11:52,overcast clouds
4,4,Severnyy,RU,50.6770,36.5532,84.24,39,0,13.42,2022-08-15 16:11:52,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input('What is the maximum temperature you would like to be in? '))

What is the maximum temperature you would like to be in? 95


In [4]:
min_temp = float(input('What is the minimum temperature you would like to be in? '))

What is the minimum temperature you would like to be in? 75


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df['Max Temp'] < max_temp) & (city_data_df['Max Temp'] > min_temp)]

In [15]:
# 4a. Determine if there are any empty rows.
new_clean_df = clean_df.copy()
new_clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Date                   0
Current Description    0
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_clean_df.replace('', np.nan, inplace=True)
new_clean_df.dropna(inplace=True)
new_clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Current Description
4,4,Severnyy,RU,50.6770,36.5532,84.24,39,0,13.42,2022-08-15 16:11:52,clear sky
5,5,Atuona,PF,-9.8000,-139.0333,77.52,65,15,20.85,2022-08-15 16:11:53,few clouds
9,9,Bontang,ID,0.1333,117.5000,78.82,85,100,7.36,2022-08-15 16:11:54,overcast clouds
10,10,Mvangue,CM,2.9667,11.5167,78.24,60,80,5.19,2022-08-15 16:11:54,broken clouds
13,13,Skegness,GB,53.1436,0.3363,79.02,62,100,6.15,2022-08-15 16:11:56,overcast clouds
...,...,...,...,...,...,...,...,...,...,...,...
671,671,Fairview,US,40.8126,-73.9990,85.53,43,0,9.22,2022-08-15 16:28:00,clear sky
674,674,Manggar,ID,-2.8833,108.2667,81.37,78,81,6.13,2022-08-15 16:30:19,broken clouds
678,678,Cananeia,BR,-25.0147,-47.9267,82.22,45,0,6.71,2022-08-15 16:30:21,clear sky
679,679,Shache,CN,38.4167,77.2406,79.84,45,83,22.59,2022-08-15 16:30:22,broken clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Severnyy,RU,84.24,clear sky,50.6770,36.5532,
5,Atuona,PF,77.52,few clouds,-9.8000,-139.0333,
9,Bontang,ID,78.82,overcast clouds,0.1333,117.5000,
10,Mvangue,CM,78.24,broken clouds,2.9667,11.5167,
13,Skegness,GB,79.02,overcast clouds,53.1436,0.3363,
19,Baiao,BR,94.71,clear sky,-2.7906,-49.6717,
28,Palmer,US,84.25,overcast clouds,42.1584,-72.3287,
33,Paka,MY,80.78,overcast clouds,4.6374,103.4368,
41,Kulevcha,UA,82.76,overcast clouds,46.0303,29.9361,
45,Dingle,PH,75.63,overcast clouds,10.9995,122.6711,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()
clean_hotel_df.replace('', np.nan, inplace=True)
clean_hotel_df.dropna(inplace=True)
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Severnyy,RU,84.24,clear sky,50.6770,36.5532,Full House North Mini-Hotel
5,Atuona,PF,77.52,few clouds,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
9,Bontang,ID,78.82,overcast clouds,0.1333,117.5000,KOPKAR Hotel Bintang Sintuk
13,Skegness,GB,79.02,overcast clouds,53.1436,0.3363,The Royal Hotel
19,Baiao,BR,94.71,clear sky,-2.7906,-49.6717,Município de Baião
...,...,...,...,...,...,...,...
668,La Romana,DO,91.76,broken clouds,18.4273,-68.9728,Casa de Campo Resort and Villas
671,Fairview,US,85.53,clear sky,40.8126,-73.9990,Homewood Suites by Hilton Edgewater-NYC Area
674,Manggar,ID,81.37,broken clouds,-2.8833,108.2667,Guest Hotel
678,Cananeia,BR,82.22,clear sky,-25.0147,-47.9267,Pousada Ancoradouro


In [11]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))